# Session analysis

In [8]:
import pandas as pd

from mna.sessions.eye_session import process_session_eye
from mna.sessions.eeg_session import process_session_eeg
from mna.sessions.motor_session import process_session_motor
from mna.sessions.ecg_session import process_session_ecg

In [9]:
import matplotlib
matplotlib.use('Agg')

# 1. Read a RN App, converted pkl file, and create the metadata and data structure

In [10]:
from os import listdir
from os.path import isfile, join
from mna.utils.rnapp_data_format import read_all_lslpresets, return_metadata_from_name, event_data_from_data
import pickle

In [11]:
data_dir = "../data/"
lsl_dir = "../mna/LSLPresets/"
onlyfiles = [f for f in listdir(data_dir) if isfile(join(data_dir, f)) and '.pkl' in f]
input_path = data_dir + onlyfiles[-1] # pick a random file
print(f"input_path {input_path}")
metadata_jsons = read_all_lslpresets(path_to_jsonfiles=lsl_dir)
with open(input_path, 'rb') as handle:
    rns_data = pickle.load(handle)
    
## Add metadata to data

for key in rns_data.keys():
    rns_data[key].append(return_metadata_from_name(key, metadata_jsons))

input_path ../data/06_15_2022_10_38_35-Exp_myexperiment-Sbj_02-Ssn_02.dats.pkl


# 2. Create new events (trial start etc.)

In [12]:
event_df = event_data_from_data(rns_data)
event_df['trial_damage'] = event_df.damage.diff().fillna(0)
event_df['trial_duration'] = event_df.trial_end_time - event_df.trial_start_time
percent_missing = event_df.notnull().sum() / len(event_df)
summary_statistics = {}
summary_statistics['voice_success_rate'] = percent_missing['voice_timestamp']
if 'chunk_timestamp' in percent_missing:
    summary_statistics['chunk_success_rate'] = percent_missing['chunk_timestamp']
else:
    summary_statistics['chunk_success_rate'] = 0

# temporary fix for pilot phase where we had some incomplete data
if 'block_condition' not in event_df:
    event_df['block_condition'] = 'practice'
    event_df.loc[5:,'block_condition'] = 'voice'

event_df['spoken_difficulty_encoded'] = event_df.spoken_difficulty.replace(to_replace=['easy', 'hard', 'unknown'],
                                                                      value=[1, 2, None])

In [13]:
rns_data.keys()

dict_keys(['Unity_MotorInput', 'Unity_HeadTracker', 'Unity_ViveSREyeTracking', 'BioSemi', 'PubSub', 'Unity_TrialInfo', 'Unity_ChunkInfo', 'AIYVoice'])

# 3. ECG Data Processing

In [14]:
post_processed_event_df = process_session_ecg(rns_data, event_df,plot_frequency=20,plot_ecg_snippet=40)

# 4. Eye Tracking Processing

In [15]:
post_processed_event_df = process_session_eye(rns_data, post_processed_event_df,detect_blink=True,plot_frequency=20, plot_eye_snippet=40)

Computed velocity exceeds threshold. Inappropriate filter setup? [2087.7 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [2566.9 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1798.8 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1811.1 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1711.0 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1991.5 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1886.0 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1862.0 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1512.8 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1002.0 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1054.9 > 1000.0 deg/s]
Computed velocity exc

# 5. EEG

In [16]:
post_processed_event_df, epochs, events, event_dict, info, reject_log, ica = process_session_eeg(rns_data, post_processed_event_df,
                                                                    event_column='spoken_difficulty_encoded', run_autoreject=True, run_ica=True)

Creating RawArray with float64 data, n_channels=89, n_times=2725394
    Range : 0 ... 2725393 =      0.000 ...  1330.758 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 55 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 55.00 Hz
- Upper transition bandwidth: 13.75 Hz (-6 dB cutoff frequency: 61.88 Hz)
- Filter length: 67585 samples (33.000 sec)

Not setting metadata
48 matching events found
Setting baseline interval to [-0.2001953125, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from p

# 5. Motor

In [17]:
post_processed_event_df = process_session_motor(rns_data, post_processed_event_df, motor_channel='Unity_MotorInput',
                                                plot_motor_result = True, plot_motor_snippet = 30, plot_frequency = 10)


# 5. Save

In [18]:
post_processed_event_df.to_csv(f"../output/ppid_{post_processed_event_df.iloc[0].ppid}_session_{post_processed_event_df.iloc[0].session}.csv")